In [1]:
import sys
import logging
import os
from pathlib import Path
from pprint import pprint as pp

# figure out the correct path
machop_path = Path(".").resolve().parent.parent /"machop"
assert machop_path.exists(), "Failed to find machop at: {}".format(machop_path)
sys.path.append(str(machop_path))

from chop.tools.checkpoint_load import load_model
from chop.dataset import MaseDataModule, get_dataset_info
from chop.tools.logger import set_logging_verbosity

from chop.passes.graph.analysis import (
    report_node_meta_param_analysis_pass,
    profile_statistics_analysis_pass,
)
from chop.passes.graph import (
    add_common_metadata_analysis_pass,
    init_metadata_analysis_pass,
    add_software_metadata_analysis_pass,
)
from chop.tools.get_input import InputGenerator
from chop.ir.graph.mase_graph import MaseGraph

from chop.models import get_model_info, get_model

set_logging_verbosity("info")


/home/agomotto3000/anaconda3/envs/mase/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO     Set logging level to info


In [2]:
batch_size = 512
model_name = "toy_convnet"
dataset_name = "cifar10"

data_module = MaseDataModule(
    name=dataset_name,
    batch_size=batch_size,
    model_name=model_name,
    num_workers=0,
    # custom_dataset_cache_path="../../chop/dataset"
)
data_module.prepare_data()
data_module.setup()

model_info = get_model_info(model_name)
model = get_model(
    model_name,
    task="cls",
    dataset_info=data_module.dataset_info,
    pretrained=False,
    checkpoint = None)

# LAB1_CUSTOM_PATH = "/home/bkt123/dev/advanced-deep-learning-systems/mase/mase_output/lab-1_jsc-custom/software/training_ckpts/best.ckpt"
# model = load_model(load_name=LAB1_CUSTOM_PATH, load_type="pl", model=model)

input_generator = InputGenerator(
    data_module=data_module,
    model_info=model_info,
    task="cls",
    which_dataloader="train",
    max_batches=1
)

dummy_in = next(iter(input_generator))
_ = model(**dummy_in)

# generate the mase graph and initialize node metadata
mg = MaseGraph(model=model)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [3]:
# from chop.actions import train
# import torch

# # print(isinstance(mg.model, torch.nn.Module))

# model = mg.model
# model_info = get_model_info('jsc-tiny')
# dataset_info = get_dataset_info('jsc')
# task = "cls"

# train_params = {
#     "model": model,
#     "model_info": model_info,
#     "data_module": data_module,
#     "dataset_info": dataset_info,
#     "task": task,
#     "optimizer": "adam",
#     "learning_rate": 1e-3,
#     "weight_decay": 0,
#     "plt_trainer_args": {
#         "max_epochs": 1,
#     }, 
#     "auto_requeue": False,
#     "save_path": None,
#     "visualizer": None,
#     "load_name": None,
#     "load_type": None
# }

# train(**train_params)

In [4]:
from pprint import pprint

from chop.passes.graph.utils import get_mase_op

mg, _ = init_metadata_analysis_pass(mg, None)
mg, _ = add_common_metadata_analysis_pass(mg, {"dummy_in": dummy_in})
mg, _ = add_software_metadata_analysis_pass(mg, None)

# pprint(mg.meta['mase'].__dict__)

for node in mg.fx_graph.nodes:
    if get_mase_op(node) == 'conv2d':
        print(node.name)
        print(50*'-')
        # print(node.meta['mase'].parameters['common']['args']['data_in_0']['value'])
        # print(node.meta['mase'].parameters['common']['args']['data_in_0']['value'])
        # print(node.meta['mase'].parameters['common']['args']['weight']['value'])
        # print(node.meta['mase'].parameters['common']['results']['data_out_0']['value'])
        pprint(mg.modules[node.target].weight)
        pprint(mg.modules[node.target].state_dict())
        print(50*'-')

block_1_0
--------------------------------------------------
Parameter containing:
tensor([[[[ 0.0406,  0.1007,  0.1421],
          [-0.0221, -0.1440, -0.0653],
          [-0.1475,  0.1540,  0.1288]],

         [[ 0.1485,  0.1614, -0.1640],
          [ 0.1323, -0.0085, -0.1097],
          [-0.1292,  0.1169,  0.0516]],

         [[ 0.1849,  0.0419, -0.1325],
          [-0.0087,  0.1468,  0.1605],
          [ 0.0924,  0.1582, -0.0117]]],


        [[[ 0.1416,  0.1599, -0.1145],
          [-0.0103,  0.1711,  0.0970],
          [-0.0603,  0.1344, -0.1182]],

         [[-0.1316, -0.0681, -0.1880],
          [-0.0262, -0.0429,  0.1686],
          [-0.1269, -0.1366, -0.1418]],

         [[ 0.0225, -0.1792,  0.1807],
          [ 0.0951,  0.0091, -0.1411],
          [ 0.0601,  0.1470, -0.1021]]],


        [[[ 0.0810, -0.1291,  0.0744],
          [ 0.1471,  0.0882, -0.0180],
          [-0.1234,  0.0954, -0.0194]],

         [[ 0.1366, -0.1777, -0.0891],
          [-0.0346,  0.1356, -0.0091],
  

In [5]:
from chop.passes.graph.transforms import (
    prune_transform_pass,
)
pass_args = {
    "weight":{
        "scope" : "local",
        "granularity" : "channel",
        "method" :  "l1-norm",
        "sparsity" : 0.5,
    },
    "activation":{
        "scope" : "local",
        "granularity" : "elementwise",
        "method" : "l1-norm",
        "sparsity" : 0.5,
    },
}
 
mg, _ = prune_transform_pass(mg, pass_args)



tried
Reached3
Mask3
tensor([[[[ True,  True,  True],
          [ True,  True,  True],
          [ True,  True,  True]],

         [[ True,  True,  True],
          [ True,  True,  True],
          [ True,  True,  True]],

         [[ True,  True,  True],
          [ True,  True,  True],
          [ True,  True,  True]]],


        [[[ True,  True,  True],
          [ True,  True,  True],
          [ True,  True,  True]],

         [[ True,  True,  True],
          [ True,  True,  True],
          [ True,  True,  True]],

         [[ True,  True,  True],
          [ True,  True,  True],
          [ True,  True,  True]]],


        [[[False, False, False],
          [False, False, False],
          [False, False, False]],

         [[False, False, False],
          [False, False, False],
          [False, False, False]],

         [[False, False, False],
          [False, False, False],
          [False, False, False]]],


        [[[ True,  True,  True],
          [ True,  True,  True]

In [12]:
from pprint import pprint

from chop.passes.graph.utils import get_mase_op

# pprint(mg.meta['mase'].__dict__)

for node in mg.fx_graph.nodes:
    if get_mase_op(node) == 'conv2d':
        print(f"Layer: {node.name}")
        # pprint(node.meta['mase'].parameters['common'])
        # pprint(node.meta['mase'].parameters['common']['args']['data_in_0']['value'])
        # pprint(node.meta['mase'].parameters['common']['args']['weight']['value'])
        pprint(mg.modules[node.target].data_out_0)
        #pprint(mg.modules[node.target].parametrizations['weight'][0].mask)
        pprint(mg.modules[node.target].state_dict())
        # pprint(node.meta['mase'].parameters['common']['results']['data_out_0']['value'])
        #print(mg.modules[node.target].parametrizations['weight'][0].mask == mg.modules[node.target].parametrizations['weight'][1].mask)
        total_w = 0
        pruned_w = 0
        w = mg.modules[node.target].weight
        for s in w:
            total_w += s.numel()
            pruned_w += s.numel() - s.nonzero().numel()

        pruned_percent = pruned_w / total_w
        print(f"Pruned percent: {pruned_percent}")

        print(50*'-')


Layer: block_1_0


AttributeError: 'ParametrizedConv2d' object has no attribute 'data_out_0'

In [7]:
# from pprint import pprint

# from chop.passes.graph.utils import get_mase_op

# # pprint(mg.meta['mase'].__dict__)

# for node in mg.fx_graph.nodes:
#     if get_mase_op(node) == 'linear':
#         print(f"Layer: {node.name}")
#         # pprint(node.meta['mase'].parameters['common'])
#         # pprint(node.meta['mase'].parameters['common']['args']['data_in_0']['value'])
#         # pprint(node.meta['mase'].parameters['common']['args']['weight']['value'])
#         pprint(mg.modules[node.target].weight)
#         # pprint(mg.modules[node.target].parametrizations['weight'][0].mask)
#         # pprint(node.meta['mase'].parameters['common']['results']['data_out_0']['value'])

#         total_w = 0
#         pruned_w = 0
#         mask_2= mg.modules[node.target].parametrizations['weight'][0].mask
#         for s in mask_2:
#             total_w += s.numel()
#             pruned_w += s.numel() - s.nonzero().numel()

#         pruned_percent = pruned_w / total_w
#         print(f"Pruned percent: {pruned_percent}")

#         print(50*'-')


In [8]:
# from chop.actions import train
# import torch

# # print(isinstance(mg.model, torch.nn.Module))

# model = mg.model
# model_info = get_model_info('jsc-tiny')
# dataset_info = get_dataset_info('jsc')
# task = "cls"

# train_params = {
#     "model": model,
#     "model_info": model_info,
#     "data_module": data_module,
#     "dataset_info": dataset_info,
#     "task": task,
#     "optimizer": "adam",
#     "learning_rate": 1e-3,
#     "weight_decay": 0,
#     "plt_trainer_args": {
#         "max_epochs": 1,
#     }, 
#     "auto_requeue": False,
#     "save_path": None,
#     "visualizer": None,
#     "load_name": None,
#     "load_type": None
# }

# train(**train_params)

In [9]:
# from pprint import pprint

# from chop.passes.graph.utils import get_mase_op


# # pprint(mg.meta['mase'].__dict__)

# for node in mg.fx_graph.nodes:
#     if get_mase_op(node) == 'linear':
#         print(node.name)
#         print(50*'-')
#         # pprint(node.meta['mase'].parameters['common'])
#         # pprint(node.meta['mase'].parameters['common']['args']['data_in_0']['value'])
#         # pprint(node.meta['mase'].parameters['common']['args']['weight']['value'])
#         pprint(mg.modules[node.target].weight)
#         pprint(mg.modules[node.target].parametrizations['weight'][0].mask)
#         # pprint(node.meta['mase'].parameters['common']['results']['data_out_0']['value'])

#         print(50*'-')